In [1]:
import json

import pandas as pd
from dotenv import load_dotenv

import sample
from llm import LLM
from solver import perform_one_token_cpc, perform_cot_cpc

load_dotenv()

True

When the ai considers whether to step back or not, does its one-word answer differ from its CoT answer?

In [2]:
passages = json.load(open("data/passages1.json"))

Each 'passage' is a lengthy text where we are reasoning through a problem.
Consider progressively larger context parts of each passage (that is, we are 'checking in' as we proceed through reading the passage), and for each context part, ask the LLM if the current approach is working or not.

In [3]:
# one-row test
llm = LLM("gpt-3.5-turbo")
for context in sample.checkpoints(passages["coding_interviews"][0], 1000):
    one_token_cpc_result = await perform_one_token_cpc(llm, context)
    (cot_cpc_thoughts, cot_cpc_result) = await perform_cot_cpc(llm, context)
    print(context)
    print(one_token_cpc_result)
    print(cot_cpc_thoughts)
    print(cot_cpc_result)
    print()

Write a program to find the length of the longest consecutive increasing sequence from an array of integers.

"Okay, for this coding interview question to find the length of the longest consecutive increasing sequence from an array of integers...let me start with a basic brute force approach of comparing each number to the next number in the array.

```python
def soln(nums):
    max_seq = 0
    curr_seq = 1
    for i in range(len(nums)-1):
        if nums[i] + 1 == nums[i+1]:
            curr_seq += 1
        else:
            curr_seq = 1
        max_seq = max(max_seq, curr_seq)
    return max_seq
```

Hmm, but stepping through this I can see it won't be very efficient for cases with large ranges of numbers, since I may need to check every single pair. There are definitely better ways than this nested loop.

Let me try sorting the array first, then I can just walk through linearly and check the differences between consecutive numbers:

```python
def soln(nums):
    nums.sort()
    max

In [4]:
async def experiment1(llm, passages):
    df = pd.DataFrame(columns=["category", "context", "one_token_cpc_result", "cot_cpc_thoughts", "cot_cpc_result"])
    for category, passages_list in passages.items():
        for passage_index, passage in enumerate(passages_list):
            print(f"Category {category}: passage {passage_index} of {len(passages_list)}...")
            for context in sample.checkpoints(passage, 1000):
                one_token_cpc_result = await perform_one_token_cpc(llm, context)
                (cot_cpc_thoughts, cot_cpc_result) = await perform_cot_cpc(llm, context)
                df = pd.concat([df, pd.DataFrame(
                    {"category": category, "context": context.text, "one_token_cpc_result": one_token_cpc_result,
                     "cot_cpc_thoughts": cot_cpc_thoughts, "cot_cpc_result": cot_cpc_result}, index=[0])],
                               ignore_index=True)
    return df

In [5]:
experiment1_gpt3 = await experiment1(LLM("gpt-3.5-turbo"), passages)
experiment1_gpt4 = await experiment1(LLM("gpt-4"), passages)

Category proofs: passage 0 of 10...


AttributeError: 'str' object has no attribute 'text'

For each result, determine whether the result is good (the two cpc methods agreed) or bad (they disagreed).

In [21]:
def score(df):
    df["score"] = df.apply(
        lambda row: 1.0 if row.one_token_cpc_result.lower() == row.cot_cpc_result.lower() else 0.0, axis=1)
    print(df["score"].mean())
    return df

In [23]:
score(experiment1_gpt3)
score(experiment1_gpt4)

0.7012987012987013
0.8311688311688312


,category,context,one_token_cpc_result,cot_cpc_thoughts,cot_cpc_result,score
0,proofs,"For this problem, I need to prove that for any...",Yes,"Yes, it may be easier to use a direct proof to...",No,0.0
1,proofs,"For this problem, I need to prove that for any...",Yes,"Yes, it seems like our current approach is not...",No,0.0
2,proofs,"For this problem, I need to prove that for any...",No,"No, there is no need to change the approach. T...",No,1.0
3,proofs,"For this problem, I need to prove that for any...",No,There is no need to change to a different appr...,No,1.0
4,proofs,"Hmm, I need to prove this statement about prim...",No,"No, there's no need to change the approach. Th...",No,1.0
...,...,...,...,...,...,...
72,coding_interviews,"For this interview question, I need to write a...",No,"No, your current approach seems sound and effi...",No,1.0
73,coding_interviews,"For this coding interview question, I need to ...",Yes,"Yes, we should use a different approach. A mor...",No,0.0
74,coding_interviews,"For this coding interview question, I need to ...",No,"No, there isn't a need to change the approach ...",No,1.0
75,coding_interviews,"For this coding interview question, I need to ...",No,The current approach is working well and shoul...,No,1.0


In [24]:
experiment1_gpt3.to_csv("experiment1_20narratives_gpt35")
experiment1_gpt4.to_csv("experiment1_20narratives_gpt4")